Combine results of generated datasets in `intermediates` and pivot the results into a point-in-time format.

The best way to track changes when we have a (start date, end date) would be to follow a point-in-time approach which takes a range of days and splits
it into day-to-day view of the data. 

**Columns for stratification:**

- Creator: creator involved
- Employee: employee assisting creator
- Department: department the employee belongs to
- Report date: day of tracking

**Columns for reporting:**

- Initiated contact: did the employee begin working for that creator that day (did they open a ticket w/ the creator) (0/1)
- Is ticket open: is the employee still working on that creator ticket (0/1)
- Ticket age: how old the ticket is in days
- Ticket closed: if the ticket closed (0/1) // If initialized contact && open contact are 0, then this should be 1 and vice-versa
- Days to close: days to close the ticket // If initialized contact && open contact are 0, then this should be >= 1, otherwise None

In [2]:
import pandas as pd

In [3]:
DEPARTMENTS = [
    "SALES",
    "CONTRACT",
    "DESIGN",
    "MANUFACTURING",
    "CAMPAIGN"
]

In [7]:
dfs = []

for i, dep in enumerate(DEPARTMENTS):
    file = f"./intermediates/{dep}.csv"
    d = pd.read_csv(file)[["Creator", "Employee", "start_date", "end_date"]]
    d["Department"] = [dep]*d.shape[0]
    dfs.append(d)

df = pd.concat(dfs)
# df.to_csv("./out/combined.csv")

In [31]:
df['start_date'] = df['start_date'].astype('datetime64[ns]')
df['end_date'] = df['end_date'].astype('datetime64[ns]')

In [32]:
df[df['end_date'].isnull()]

,Creator,Employee,start_date,end_date,Department
647,Michael Louis,Bruce Gulley,2022-09-21 20:37:01,NaT,SALES
648,Walter Leonberger,Jane Richardson,2022-09-22 00:33:34,NaT,SALES
649,Dan Karlin,Fredrick Elfrink,2022-09-27 05:41:37,NaT,SALES
650,Roy Mathis,Gilberto Delossantos,2022-10-03 14:35:55,NaT,SALES
651,James Schrandt,Cassandra Salamanca,2022-10-03 15:52:24,NaT,SALES
...,...,...,...,...,...
529,Danielle Chiodi,Adriana Coleman,2022-03-12 09:45:26,NaT,CAMPAIGN
530,Timothy Ward,John Gillispie,2022-03-13 15:24:20,NaT,CAMPAIGN
531,Charles Meidinger,Adriana Coleman,2022-03-14 17:41:10,NaT,CAMPAIGN
532,Gerald Taylor,Rose Churchill,2022-03-17 21:04:58,NaT,CAMPAIGN


In [40]:

def get_point_in_time_metrics(row):
    """
    Note:
        If start_date == end_date, then it took less than a day
            and the information is completely ignored
        
        The days_to_close is the number of days from start_date
            to end_date - 1 since we don't include end_date.
        
        If row['end_date'] is None, then the range should be today()
            and ticket_closed and days_to_close == 0
    """
    out_rows = []
    if row['start_date'] == row['end_date']:
        return out_rows

    end_date = row['end_date']
    if end_date != end_date: # Way of checking if value is null
        end_date = pd.Timestamp.now()

    dates = pd.date_range(row['start_date'], end_date)
    for ticket_age, date in enumerate(dates):
        point_in_time_data = dict(
            report_date=date,
            creator=row['Creator'],
            department=row['Department'],
            employee=row['Employee'],
            start_date=row['start_date'], # For DEBUGGING
            end_date=row['end_date'], # For DEBUGGING
            initiated_contact=int(date == row['start_date']),
            is_ticket_open=int(date != row['end_date']),
            ticket_age=ticket_age if date != row['end_date'] else None,

            ticket_closed=int(date == row['end_date']),
            days_to_close=len(dates) - 1 if date == row['end_date'] else None
        )

        out_rows.append(point_in_time_data)

    return out_rows

point_in_time_rows = []
for _, row in df.iterrows():
    rows_to_add = get_point_in_time_metrics(row)
    point_in_time_rows += rows_to_add

pita = pd.DataFrame(point_in_time_rows)
pita.to_csv("./out/point_in_time_metrics.csv")

In [41]:
pita

,report_date,creator,department,employee,start_date,end_date,initiated_contact,is_ticket_open,ticket_age,ticket_closed,days_to_close
0,2019-01-02 21:49:39,Patti Martinez,SALES,Cassandra Salamanca,2019-01-02 21:49:39,2019-01-14 21:49:39,1,1,0.0,0,NaN
1,2019-01-03 21:49:39,Patti Martinez,SALES,Cassandra Salamanca,2019-01-02 21:49:39,2019-01-14 21:49:39,0,1,1.0,0,NaN
2,2019-01-04 21:49:39,Patti Martinez,SALES,Cassandra Salamanca,2019-01-02 21:49:39,2019-01-14 21:49:39,0,1,2.0,0,NaN
3,2019-01-05 21:49:39,Patti Martinez,SALES,Cassandra Salamanca,2019-01-02 21:49:39,2019-01-14 21:49:39,0,1,3.0,0,NaN
4,2019-01-06 21:49:39,Patti Martinez,SALES,Cassandra Salamanca,2019-01-02 21:49:39,2019-01-14 21:49:39,0,1,4.0,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...
95095,2023-01-08 02:59:28,Emma Keane,CAMPAIGN,Rose Churchill,2022-03-25 02:59:28,NaT,0,1,289.0,0,NaN
95096,2023-01-09 02:59:28,Emma Keane,CAMPAIGN,Rose Churchill,2022-03-25 02:59:28,NaT,0,1,290.0,0,NaN
95097,2023-01-10 02:59:28,Emma Keane,CAMPAIGN,Rose Churchill,2022-03-25 02:59:28,NaT,0,1,291.0,0,NaN
95098,2023-01-11 02:59:28,Emma Keane,CAMPAIGN,Rose Churchill,2022-03-25 02:59:28,NaT,0,1,292.0,0,NaN
